In [ ]:
!pip install openai
!pip install rpy2
!pip install google-cloud-secret-manager

In [ ]:
from IPython import get_ipython
from IPython.display import Markdown, display

In [ ]:
from google.cloud import secretmanager

def access_secret_version(resource_name):
    client = secretmanager.SecretManagerServiceClient()
    response = client.access_secret_version(request={"name": resource_name})
    return response.payload.data.decode("UTF-8")

# Your secret's resource name
resource_name = "projects/286751827882/secrets/openai_key/versions/1"

# Access the secret
openai_key = access_secret_version(resource_name)



In [ ]:
from openai import OpenAI
client = OpenAI(api_key=openai_key)

In [ ]:
def explain(cell_number):
    """Return the content of the specified cell number."""
    ipython = get_ipython()  # Get the current IPython instance
    
    # Ensure the cell number is valid
    if cell_number < len(ipython.user_ns['In']):
        response = client.chat.completions.create(
          model="gpt-4o-mini",
          messages=[
        {
            "role": "system",
            "content": "You are a helpful coding assitant providing advice to someone with limited coding experience. Please format your response in markdown "
        },
        {
          "role": "user",
          "content": "Please concisely comment this code to explain each line. Ignore the run cell magic and just focus on the R code " + ipython.user_ns['In'][cell_number]
        }
      ],
      temperature=1,
      max_tokens=2204,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
        )

        #print(response.choices[0].message.content)
        return display(Markdown(response.choices[0].message.content))
        # Return the content of the cell
        #return ipython.user_ns['In'][cell_number]
    else:
        # Error message for invalid cell number
        return "Cell number out of range."
    

    
def modify(cell_number,modification):
    """Return the content of the specified cell number."""
    ipython = get_ipython()  # Get the current IPython instance
    
    # Ensure the cell number is valid
    if cell_number < len(ipython.user_ns['In']):
        response = client.chat.completions.create(
          model="gpt-4o-mini",
          messages=[
        {
            "role": "system",
            "content": "You are a helpful coding assitant. Please only provide R code "
        },
        {
          "role": "user",
          "content": "Please modify this R code " + ipython.user_ns['In'][cell_number] + " to accomplish " + modification + "Ignore the run cell magic and just focus on the R code. All of the tidyverse libraries are already loaded. You do not need to load any libraries. Return only code." 
        },
        {
          "role": "assistant",
          "content": "top_downregulated_genes <- gene %>% \n\t filter(logFC < 0) %>% \n\t arrange(desc(adj.P.Val)) %>% \n\t slice_head(n = 5) \n\n # View the results \n print(top_downregulated_genes)" 
        }      
      ],
      temperature=1,
      max_tokens=2204,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
        )

        #print(response.choices[0].message.content)
        #return display(Markdown(response.choices[0].message.content))
        #return create_new_cell(response.choices[0].message.content)
        return create_new_cell("%%R\n\n" + response.choices[0].message.content)

        # Return the content of the cell
        #return ipython.user_ns['In'][cell_number]
    else:
        # Error message for invalid cell number
        return "Cell number out of range."

def propose(suggest):
        response = client.chat.completions.create(
          model="gpt-4o-mini",
          messages=[
        {
            "role": "system",
            "content": "You are a helpful coding assitant. Please only provide R code with no markdown"
        },
        {
          "role": "user",
          "content": "Please suggest R code to accomplish " + suggest + ".  If possible always suggest tidyverse code. All of the tidyverse libraries are already loaded. You do not need to load any libraries. Return only code, do not use markdown." 
        },
        {
          "role": "assistant",
          "content": "top_downregulated_genes <- gene %>% \n\t filter(logFC < 0) %>% \n\t arrange(desc(adj.P.Val)) %>% \n\t slice_head(n = 5) \n\n # View the results \n print(top_downregulated_genes)" 
        }      
      ],
      temperature=1,
      max_tokens=2204,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
        )

        #print(response.choices[0].message.content)
        #return display(Markdown(response.choices[0].message.content))
        return create_new_cell("%%R\n\n" + response.choices[0].message.content)
    
def create_new_cell(contents):
    shell = get_ipython()
    shell.set_next_input(contents, replace=False)


In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R

install.packages("tidyverse")
install.packages("palmerpenguins")
install.packages("ggthemes")
library(tidyverse)

library(palmerpenguins)
library(ggthemes)

In [ ]:
%%R

gene<-read_tsv('limma-voom_luminalpregnant-luminallactate')


In [ ]:
%%R

top_upregulated_genes <- gene %>%
  filter(logFC > 0) %>%
  arrange(adj.P.Val) %>%
  slice_head(n = 10)

# View the results
print(top_upregulated_genes)

In [ ]:
explain(13)

In [ ]:
propose("I need to find the 25 most signficantly upregulated genes from a dataframe called gene. \
Here are the column names: ENTREZID, SYMBOL, GENENAME, logFC AveExpr, t, P.Value, adj.P.Val")

In [ ]:
%%R

top_upregulated_genes <- gene %>% 
  filter(logFC > 0) %>% 
  arrange(adj.P.Val) %>% 
  slice_head(n = 25) 

print(top_upregulated_genes)

In [ ]:
%%R

top_upregulated_genes <- gene %>% 
  arrange(desc(logFC)) %>% 
  slice_head(n = 25) 

In [ ]:
%%R

ggplot(
  data = penguins,
  mapping = aes(x = flipper_length_mm, y = body_mass_g)
) +
  geom_point(aes(color = species, shape = species)) +
  geom_smooth(method = "lm") +
  labs(
    title = "Body mass and flipper length",
    subtitle = "Dimensions for Adelie, Chinstrap, and Gentoo Penguins",
    x = "Flipper length (mm)", y = "Body mass (g)",
    color = "Species", shape = "Species"
  ) +
  scale_color_colorblind() +
  theme_gray(base_size = 12)

In [ ]:
explain(15)

In [ ]:
modify(15,"remove gentoo penguins from the graph as they are my least favorite penguin")

In [ ]:
%%R


ggplot(
  data = penguins %>% filter(species != "Gentoo"),
  mapping = aes(x = flipper_length_mm, y = body_mass_g)
) +
  geom_point(aes(color = species, shape = species)) +
  geom_smooth(method = "lm") +
  labs(
    title = "Body mass and flipper length",
    subtitle = "Dimensions for Adelie and Chinstrap Penguins",
    x = "Flipper length (mm)", 
    y = "Body mass (g)",
    color = "Species", 
    shape = "Species"
  ) +
  scale_color_colorblind() +
  theme_gray(base_size = 12)
